# Introduction

This notebook is purely for development. When we want to create a new model, we just use this notebook, pull in data from somewhere (ideally generated from our `simulator`) and train a model. 

When we finish, we will convert this notebook to a script, that we execute (reading data from the same file, training the model, etc) as soon as the docker container starts. The file dumps a model that our server.py loads and uses to perform predictions against data coming in via POST requests from our simulator.


### Overview on the anomaly prediction system 

This model is trained on signals that are in a healthy state. It's the ground truth of our system. When we pass in a new waveform, our model will predict the expected waveform and do a validation against the prediction vs what we passed in. If we encounter a deviation from our input to what the model has predicted the waveform to behave like, we flag the input as anomalous, scaled in criticality based on a threshold. Ie, 10% error dictates a high level of risk for the business.

In [138]:
import matplotlib.pyplot as plt
import numpy as np 
import tensorflow as tf 
import csv
import json

DEBUG = False

## Loading our data

In [139]:
!head readings.csv

reading_type,reading
normal,"[3.351600230178196, 3.4192233870403532, 3.4867716922342296, 3.5542239082609455, 3.6215587954986295, 3.688755123593989, 3.7557916828538644, 3.8226472956299555, 3.889300827689874, 3.955731199567716, 4.02191739788735, 4.087838486651604, 4.153473618490594, 4.218802045862439, 4.283803132199623, 4.34845636299431, 4.412741356815967, 4.476637876254646, 4.540125838783375, 4.603185327533101, 4.665796601973718, 4.727940108494756, 4.789596490879365, 4.850746600665283, 4.911371507386572, 4.971452508689936, 5.03097114031956, 5.089909185964439, 5.14824868696227, 5.205971951854103, 5.263061565783933, 5.3195003997376595, 5.375271619615784, 5.430358695134424, 5.484745408549281, 5.538415863197317, 5.591354491850996, 5.643546064880076, 5.6949756982160435, 5.7456288611144055, 5.795491383710178, 5.844549464362028, 5.892789676780684, 5.940198976937312, 5.98676470974773, 6.0324746155284705, 6.077316836220801, 6.121279921379006, 6.164352833919331, 6.206524955626192, 6.2477860924123

In [140]:
def load_data(filename): 
    """
    Load our .csv file
    returns two arrays, one for the time-steps
    and the second for the actual readings at those steps.
    """
    time_steps = []
    amplitudes = []
    with open(filename, mode='r') as file: 
        csv_reader = csv.reader(file, delimiter=',')
        next(csv_reader)
        for row in csv_reader: 
            raw_list = json.loads(row[1])
            for i, value in enumerate(raw_list): 
                amplitudes.append(value)
                time_steps.append(i)
            # we're only reading one row so we break here. 
            break 
    time = np.array(time_steps)
    amps = np.array(amplitudes)
    return time, amps
time, series = load_data("readings.csv")

In [141]:
def plot_values(debug=False, series=None): 
    if debug == True: 
        plt.plot(series, label="time values")
        plt.show()

def plot_series(x, y, format="-", start=0, end=None, title=None, xlabel=None, ylabel=None, legend=None): 
    if DEBUG == True: 
        plt.figure(figsize=(10, 6))
        if type(y) is tuple: 
            for y_curr in y: 
                plt.plot(x[start:end], y_curr[start:end], format)
        else: 
            plt.plot(x[start:end], y[start:end], format)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        if legend: 
            plt.legend(legend)
        plt.title(title)
        plt.grid(True)
        plt.show()

plot_series(time, series)

## Splitting our dataset

Here we start to split our dataset into train and validation sets. We'll follow through with building out windows of these datasets to pass into a tuning model, so as to find our optimal learning rate.

In [142]:
split_time = 3000 # out of a 4k long list of values.
time_train = time[:split_time]
x_train = series[:split_time]

time_valid = time[split_time:]
x_valid = series[split_time:]

print(f"time validation set shape: {time_valid.shape}")
print(f"time train set shape: {time_train.shape}")


time validation set shape: (1001,)
time train set shape: (3000,)


#### Buffering our window sizes. 

In [143]:

from helpers import window
""" 
def window(series, window_size, batch_size, shuffle_buffer): 
    dataset = tf.data.Dataset.from_tensor_slices(series) # create the dataset frame
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True) # build out windows of a set size and shift by one for each 
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)) # batches each window into concrete arrays
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache().prefetch(1)
    return dataset
"""

' \ndef window(series, window_size, batch_size, shuffle_buffer): \n    dataset = tf.data.Dataset.from_tensor_slices(series) # create the dataset frame\n    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True) # build out windows of a set size and shift by one for each \n    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)) # batches each window into concrete arrays\n    dataset = dataset.map(lambda window: (window[:-1], window[-1]))\n    dataset = dataset.shuffle(shuffle_buffer)\n    dataset = dataset.batch(batch_size)\n    dataset = dataset.cache().prefetch(1)\n    return dataset\n'

In [144]:
window_size = 30 
batch_size = 32 
shuffle = 1000
train_set = window(x_train, window_size, batch_size=batch_size, shuffle_buffer=shuffle)

### Building our tuning model 

In [145]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(window_size,)), 
    tf.keras.layers.Dense(30, activation="relu"), 
    tf.keras.layers.Dense(30, activation="relu"), 
    tf.keras.layers.Dense(1)
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 30)                930       
                                                                 
 dense_4 (Dense)             (None, 30)                930       
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,891
Trainable params: 1,891
Non-trainable params: 0
_________________________________________________________________


In [146]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20 ))
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

Epoch 1/100


2025-05-29 20:26:15.482124: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [3000]
	 [[{{node Placeholder/_0}}]]
2025-05-29 20:26:15.482317: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [3000]
	 [[{{node Placeholder/_0}}]]


93/93 [==============================] - 0s 2ms/step - loss: 3.8235 - lr: 1.0000e-08
Epoch 2/100
93/93 [==============================] - 0s 390us/step - loss: 3.8232 - lr: 1.1220e-08
Epoch 3/100
93/93 [==============================] - 0s 375us/step - loss: 3.8229 - lr: 1.2589e-08
Epoch 4/100
93/93 [==============================] - 0s 377us/step - loss: 3.8226 - lr: 1.4125e-08
Epoch 5/100
93/93 [==============================] - 0s 364us/step - loss: 3.8222 - lr: 1.5849e-08
Epoch 6/100
93/93 [==============================] - 0s 361us/step - loss: 3.8217 - lr: 1.7783e-08
Epoch 7/100
93/93 [==============================] - 0s 367us/step - loss: 3.8212 - lr: 1.9953e-08
Epoch 8/100
93/93 [==============================] - 0s 367us/step - loss: 3.8206 - lr: 2.2387e-08
Epoch 9/100
93/93 [==============================] - 0s 364us/step - loss: 3.8199 - lr: 2.5119e-08
Epoch 10/100
93/93 [==============================] - 0s 370us/step - loss: 3.8191 - lr: 2.8184e-08
Epoch 11/100
93/93 [===

In [147]:
if DEBUG == True: 
    lrs = 1e-8 * (10 ** (np.arange(100) / 20))
    plt.figure(figsize=(10, 6))
    plt.grid(True)
    plt.semilogx(lrs, history.history['loss'])
    plt.tick_params('both', length=10, width=1, which='both')
    plt.axis([1e-8, 1e-3, 0, 100])

#### Train a new model 

In [148]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(window_size,)), 
    tf.keras.layers.Dense(30, activation='relu'), 
    tf.keras.layers.Dense(30, activation='relu'), 
    tf.keras.layers.Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                930       
                                                                 
 dense_1 (Dense)             (None, 30)                930       
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,891
Trainable params: 1,891
Non-trainable params: 0
_________________________________________________________________


In [149]:
learning_rate = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=['mae'])
history = model.fit(train_set, epochs=30)

Epoch 1/30
93/93 [==============================] - 0s 523us/step - loss: 1.2730 - mae: 1.6768
Epoch 2/30
93/93 [==============================] - 0s 414us/step - loss: 0.1437 - mae: 0.3868
Epoch 3/30
93/93 [==============================] - 0s 406us/step - loss: 0.0980 - mae: 0.3025
Epoch 4/30
93/93 [==============================] - 0s 413us/step - loss: 0.0836 - mae: 0.2729
Epoch 5/30
93/93 [==============================] - 0s 389us/step - loss: 0.0722 - mae: 0.2463
Epoch 6/30
93/93 [==============================] - 0s 417us/step - loss: 0.0616 - mae: 0.2196
Epoch 7/30
93/93 [==============================] - 0s 404us/step - loss: 0.0519 - mae: 0.1931
Epoch 8/30
93/93 [==============================] - 0s 443us/step - loss: 0.0434 - mae: 0.1675
Epoch 9/30
93/93 [==============================] - 0s 405us/step - loss: 0.0358 - mae: 0.1423
Epoch 10/30
93/93 [==============================] - 0s 417us/step - loss: 0.0292 - mae: 0.1147
Epoch 11/30
93/93 [==============================

### Build out the model predictions

In [156]:
from helpers import model_forecast
""" 
def model_forecast(model, series, window_size, batch_size): 
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda w: w.batch(window_size))
    dataset = dataset.batch(batch_size).prefetch(1)
    forecast = model.predict(dataset, verbose=0)
    return forecast
"""

' \ndef model_forecast(model, series, window_size, batch_size): \n    dataset = tf.data.Dataset.from_tensor_slices(series)\n    dataset = dataset.window(window_size, shift=1, drop_remainder=True)\n    dataset = dataset.flat_map(lambda w: w.batch(window_size))\n    dataset = dataset.batch(batch_size).prefetch(1)\n    forecast = model.predict(dataset, verbose=0)\n    return forecast\n'

In [151]:
forecast_series = series[split_time - window_size: - 1] # get the forecast series for the "validation set"
forecast = model_forecast(
    model, forecast_series, window_size, 
    batch_size) 
results = forecast.squeeze() 
plot_series(time_valid, (x_valid, results))

2025-05-29 20:26:20.911842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [1030]
	 [[{{node Placeholder/_0}}]]
2025-05-29 20:26:20.911987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [1030]
	 [[{{node Placeholder/_0}}]]


In [152]:
print("==== Final Results =====")
print(tf.keras.metrics.mse(x_valid, results).numpy())
print(tf.keras.metrics.mae(x_valid, results).numpy())
print("========================")

==== Final Results =====
0.050428163
0.04706096


In [153]:
if DEBUG: 
    loss = history.history['loss']
    loss = loss[15:]
    plt.plot(loss)
    plt.show()

### Dumping the model 

If our model meets certain criteria, then we dump it to a file we load from the server.

In [154]:
mae = tf.keras.metrics.mae(x_valid, results).numpy()
mse = tf.keras.metrics.mse(x_valid, results).numpy()
if mae <= 0.1 and mse <= 0.1: 
    print("saving model. all scores passed...")
    model.save('final.h5')

saving model. all scores passed...


In [155]:

"""

import numpy as np
import tensorflow as tf

INPUT_VALUE = payload.json()  # make sure this is a NumPy array or convertible

# load the model
model = tf.keras.models.load_model('final.h5')

# convert input to np array and reshape if needed
input_array = np.array(INPUT_VALUE)

# predict
predictions = model.predict(input_array).flatten()

# calculate errors
errors = np.abs(input_array.flatten() - predictions)

# define threshold
threshold = np.mean(errors) + 3 * np.std(errors)

# flag anomalies
anomalies = errors > threshold

# calculate anomaly percentage
anomaly_percent = np.mean(anomalies) * 100

if anomaly_percent > 5:  # example threshold, adjust as needed
    print(f"Anomaly detected! {anomaly_percent:.2f}% of points are anomalous.")
else:
    print(f"No significant anomalies detected. Only {anomaly_percent:.2f}% anomalous.")


"""

'\n\nimport numpy as np\nimport tensorflow as tf\n\nINPUT_VALUE = payload.json()  # make sure this is a NumPy array or convertible\n\n# load the model\nmodel = tf.keras.models.load_model(\'final.h5\')\n\n# convert input to np array and reshape if needed\ninput_array = np.array(INPUT_VALUE)\n\n# predict\npredictions = model.predict(input_array).flatten()\n\n# calculate errors\nerrors = np.abs(input_array.flatten() - predictions)\n\n# define threshold\nthreshold = np.mean(errors) + 3 * np.std(errors)\n\n# flag anomalies\nanomalies = errors > threshold\n\n# calculate anomaly percentage\nanomaly_percent = np.mean(anomalies) * 100\n\nif anomaly_percent > 5:  # example threshold, adjust as needed\n    print(f"Anomaly detected! {anomaly_percent:.2f}% of points are anomalous.")\nelse:\n    print(f"No significant anomalies detected. Only {anomaly_percent:.2f}% anomalous.")\n\n\n'